# Recolha da sequência de referência.

A sequência foi recolhida da base de dados *Nucleotide* e foi guardada num ficheiro de formato GeneBank. Porem nesta base de dados não foi possivél encontrar um sequência referente ao gene de interesse, sendo como tal necessário recolher desta a partir da sequência inteira do virus, sendo a sequência escolhida a "MN908947.3".

In [3]:
import os
from Bio import SeqIO, Entrez

a = %pwd
wd = (a.rsplit('/',2))[0]

seq_id = "MN908947.3"

In [4]:
handle = Entrez.efetch(db="nucleotide",
                       id=seq_id,
                       rettype="gb",
                       retmode="text")

file_path = os.path.join(wd,"data/reference/ORF7a", f"{seq_id}.gb")
with open(file_path, 'w') as file:
    file.write(handle.read())

In [4]:
path = os.path.join(wd, "data/reference/ORF7a",f"{seq_id}.gb")
seq_r = SeqIO.read(path, format="genbank")
for Key, Value in seq_r.annotations.items():
    print(Key,": ", Value)


molecule_type :  ss-RNA
topology :  linear
data_file_division :  VRL
date :  18-MAR-2020
accessions :  ['MN908947']
sequence_version :  3
keywords :  ['']
source :  Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
organism :  Severe acute respiratory syndrome coronavirus 2
taxonomy :  ['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']
references :  [Reference(title='A new coronavirus associated with human respiratory disease in China', ...), Reference(title='Direct Submission', ...)]
comment :  On Jan 17, 2020 this sequence version replaced MN908947.2.
structured_comment :  OrderedDict([('Assembly-Data', OrderedDict([('Assembly Method', 'Megahit v. V1.1.3'), ('Sequencing Technology', 'Illumina')]))])


O ficheiro .gb contêm varias, entre as quais podemos confirmar o organismo, a taxonomia do mesmo e referências bibliográficas associadas a esta sequência. Porém a existência de *features* vai permitir conhecer melhor a sequência e determinar a localização do nosso gene de interesse.

## Features

In [7]:
interest_features = []

for feature in seq_r.features:
    if feature.type in ["gene", "CDS"]:
        if feature.qualifiers["gene"][0] == "ORF7a":
            interest_features.append(feature)
            print(feature.type)
            print(feature.location)
            print(feature.ref_db)
            for Key,Value in feature.qualifiers.items():
                print(Key,": ", Value)
            print(" ")

gene
[27393:27759](+)
None
gene :  ['ORF7a']
 
CDS
[27393:27759](+)
None
gene :  ['ORF7a']
codon_start :  ['1']
product :  ['ORF7a protein']
protein_id :  ['QHD43421.1']
translation :  ['MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE']
 


Realizando a filtração destes features foi possivel encontrar tanto uma anotação para o gene como a CDS que este representa, ambas indicam um localização coincidente, que poderá ser usada para *slicing* deste gene.


## Slicing do gene de interesse.

In [29]:
from Bio.SeqRecord import SeqRecord


ORF7a_seq = seq_r.seq[interest_features[0].location.nofuzzy_start:interest_features[0].location.nofuzzy_end]
ORF7a = SeqRecord(ORF7a_seq, f"{seq_id} | Reference sequence ORF7a gene", "", "")
SeqIO.write(ORF7a, os.path.join(wd,"data/reference/ORF7a", f"{seq_id}_ORF7a.fasta"), "fasta")

1

### Extração de informação relativa à proteina

Podemos ficar a conhecer melhor a proteina, seja a identificação da ORF que permite obter a sua tradução, como a sequência desta e mais importante um ID, que poderá ser usado para pesquisar noutras base de dados como o Uniprot.

In [37]:
from Bio.Seq import Seq

ORF7a_prot_seq = Seq(interest_features[1].qualifiers['translation'][0])
ORF7a_prot = SeqRecord(ORF7a_prot_seq, interest_features[1].qualifiers['protein_id'][0] ,interest_features[1].qualifiers['product'][0], interest_features[1].qualifiers['product'][0])
SeqIO.write(ORF7a_prot, os.path.join(wd,"data/reference/ORF7a", f"{seq_id}_ORF7a_prot.fasta"), "fasta")

1

## Extração de conhecimento do Uniprot

Como referido anterirormente, com base no identificador para a proteina, foi possivel encontrar na *Uniprot* o id neste base de dados a qual esta proteina corresponde, nomeadamente P0DTC7.

Descarregando o ficheiro XML e abrindo com o SeqIO podemos então analisar a função desta proteina e algumas das suas anotações e features.

In [39]:
from wget import download

url = "https://www.ebi.ac.uk/uniprot/api/covid-19/uniprotkb/accession/P0DTC7.xml"
path = os.path.join(wd, "data/reference/ORF7a","P0DTC7_uniprot.xml")

download(url, path)
prot_ref = SeqIO.read(path, format="uniprot-xml")